In [1]:

import os
import pandas as pd
import gcd_table_sequential
import gcd_table
from convert_table_to_image import convert_styled_table_to_image

In [2]:
# # example from:
# #  https://win-vector.com/2024/10/06/dudeneys-remainder-problem/
# soln = gcd_table_sequential.build_gcd_table_filled(
#     508811 - 480608, 
#     723217 - 480608,
#     verbose=True)

# example from:
#  https://win-vector.com/2024/11/01/the-monkey-and-the-coconuts-an-introduction-to-the-extended-euclidean-algorithm/
soln = gcd_table.build_gcd_table_filled(
    15625, 
    2,
    verbose=True)

soln

,a,b,a%b,a//b,"GCD(a, b)",u,v
0,15625,2,1,7812,1,1,-7812
1,2,1,0,2,1,0,1
2,1,0,N/A,N/A,1,1,0


In [3]:

tables = list(soln.attrs['captured_tables'])  # new list, so we can append without side effects



In [4]:
# convert tables to images
for i, t in enumerate(tables):
    file_path = f'dataframe_{i:04d}.png'
    convert_styled_table_to_image(t, file_path)



Then convert at the command line with:

```
ffmpeg -framerate 0.5 -i dataframe_%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p MonkeyCoco.mp4
```


In [5]:
!ffmpeg -framerate 0.5 -i dataframe_%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p MonkeyCoco.mp4

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --

In [6]:
# clean up
for i, t in enumerate(tables):
    file_path = f'dataframe_{i:04d}.png'
    os.remove(file_path)